# Lesson 3: OpenAI Function Calling In LangChain

In [ ]:
import os
import openai
from dotenv import load_dotenv

load_dotenv()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

# Pydantic Syntax

In [ ]:
class User:
    def __init__(self,name:str,age:int,email:str):
        self.name=name
        self.age=age
        self.email=email
    

In [ ]:
foo=User(name="Joe",age=32,email="joe@gmail.com")

In [ ]:
foo.name

In [ ]:
foo=User(name="Joe",age="bar",email="joe@gmail.com")
foo.age

In [ ]:
from typing import List
from pydantic import BaseModel, Field

class pUser(BaseModel):
    name:str
    age:int
    email:str

In [ ]:
foo_p=pUser(name="Joe",age=32,email="joe@gmail.com")
foo_p.name

In [ ]:
foo_p=pUser(name="Joe",age="55",email="joe@gmail.com")
foo_p.age

In [ ]:
class Class(BaseModel):
    students:List[pUser]

In [ ]:
obj=Class(students=[pUser(name="Joe",age=32,email="joe@gmail")])

obj

# Pydantic to OpenAI function definition

In [ ]:
class WeatherSearch(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str = Field(description="airport code to get weather for")

In [ ]:
from langchain_core.utils.function_calling import convert_to_openai_function

weather_function=convert_to_openai_function(WeatherSearch)

weather_function

In [ ]:
class WeatherSearch2(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str

In [ ]:
convert_to_openai_function(WeatherSearch2)

In [ ]:
from langchain_openai import ChatOpenAI

model=ChatOpenAI()

In [ ]:

model.invoke("what is the weather in SF today?", functions=[weather_function])


In [ ]:
model_with_function=model.bind(functions=[weather_function])

In [ ]:
model_with_function.invoke("what is the weather in sf")

# Forcing it to use a function

In [ ]:
model_with_forced_function = model.bind(functions=[weather_function], function_call={"name":"WeatherSearch"})

In [ ]:
model_with_forced_function.invoke("what is the weather in sf?")

In [ ]:
model_with_forced_function.invoke("hi!")

# Using in a chain

In [ ]:
from langchain.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_messages([("system","You are a helpful assistant"),
("user","{input}")])

chain=prompt|model_with_function
chain.invoke({"input": "what is the weather in sf?"})

# Using multiple functions

In [ ]:
class ArtistSearch(BaseModel):
    """Call this to get the names of songs by a particular artist"""
    artist_name: str = Field(description="name of artist to look up")
    n: int = Field(description="number of results")

In [ ]:
functions = [
    convert_to_openai_function(WeatherSearch),
    convert_to_openai_function(ArtistSearch),
]

In [ ]:
model_with_functions=model.bind(functions=functions)
model_with_functions.invoke("what is the weather in sf?")

In [ ]:
model_with_functions.invoke("what are three songs by taylor swift?")

In [ ]:
model_with_functions.invoke("hi!")